In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from statsmodels.stats.outliers_influence import variance_inflation_factor
import lightgbm as lgb
import optuna
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report

In [1]:
df = pd.read_pickle('My Data.pkl')
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4129 entries, 2009-01-02 to 2025-06-02
Columns: 458 entries, BDI to curvature
dtypes: float64(450), int32(5), int64(3)
memory usage: 14.4 MB


In [2]:
df['month'] = df['month'].astype('category')
df['Recession'] = df['Recession'].astype('category')
df['steepen'] = df['steepen'].astype('category')
df['curvature'] = df['curvature'].astype('category')
df['steepen_2week'] = df['steepen_2week'].astype('category')
df['curvature_2week'] = df['curvature_2week'].astype('category')

df.drop(['infl_a_reverse','infl_m_reverse','infl_q_reverse'], axis=1, inplace=True)

<font size=4, font color='blue'> Rolling window percentile scaler

In [3]:
window = int(2 * 252)

to_transform = df.select_dtypes(include='number').columns

df_pct = df.copy()
for col in to_transform:
    df_pct[col] = (
        df[col]
        .rolling(window=window, min_periods=1)
        .apply(lambda x: pd.Series(x).rank(pct=True).iloc[-1], raw=False)
    )

df_pct = df_pct.loc['2010-01-01':'2025-05-30']

df_pct.info()


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3876 entries, 2010-01-04 to 2025-05-30
Columns: 455 entries, BDI to curvature
dtypes: category(6), float64(449)
memory usage: 13.3 MB


In [4]:
df_pct

,BDI,BDI_7,BDI_14,BDI_21,BDI_pct7,BDI_pct14,BDI_pct21,BDI_42,BDI_mean7,BDI_mean14,...,SP400_ret_std14,SP400_pct21,SP400_ret_std21,SP400_pct42,SP400_ret_std42,month,steepen_2week,curvature_2week,steepen,curvature
2010-01-04,0.687747,0.766798,0.916996,0.948617,0.213439,0.098814,0.122530,0.774704,0.667984,0.762846,...,0.047431,0.644269,0.015810,0.731225,0.007905,1,1,0,1,0
2010-01-05,0.720472,0.740157,0.889764,0.921260,0.370079,0.192913,0.165354,0.789370,0.657480,0.732283,...,0.027559,0.740157,0.011811,0.692913,0.007874,1,1,0,1,0
2010-01-06,0.717647,0.705882,0.870588,0.909804,0.474510,0.223529,0.184314,0.819608,0.658824,0.709804,...,0.023529,0.694118,0.003922,0.733333,0.007843,1,0,0,0,0
2010-01-07,0.683594,0.679688,0.832031,0.902344,0.457031,0.222656,0.164062,0.828125,0.652344,0.679688,...,0.023438,0.718750,0.003906,0.652344,0.003906,1,0,0,1,0
2010-01-08,0.678988,0.665370,0.817121,0.910506,0.533074,0.249027,0.155642,0.832685,0.665370,0.673152,...,0.011673,0.793774,0.003891,0.688716,0.003891,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-05-23,0.321429,0.281746,0.371032,0.291667,0.666667,0.408730,0.581349,0.544643,0.319444,0.297619,...,0.825397,0.769841,0.753968,0.299603,0.962302,5,0,1,1,1
2025-05-27,0.301587,0.309524,0.361111,0.303571,0.454365,0.375000,0.462302,0.549603,0.319444,0.289683,...,0.902778,0.910714,0.823413,0.507937,0.978175,5,0,1,1,1
2025-05-28,0.309524,0.355159,0.324405,0.313492,0.319444,0.416667,0.450397,0.539683,0.311508,0.287698,...,0.924603,0.837302,0.849206,0.456349,0.982143,5,0,0,1,0
2025-05-29,0.334325,0.323413,0.303571,0.343254,0.484127,0.539683,0.476190,0.530754,0.313492,0.291667,...,0.918651,0.817460,0.847222,0.634921,0.964286,5,0,0,1,0


In [5]:
df_pct.to_pickle('My Data Percentile.pkl')

<font size=4, font color='blue'> 8:2 Train-test split, no shuffling

In [6]:
target = 'steepen'
X = df_pct.drop(columns=[target,'curvature','steepen_2week','curvature_2week'])
y = df_pct[target]

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    shuffle=False
)

<font size=4, font color='blue'> Use correlation matrix to find out strongly-correlated feature pairs 

In [7]:
corr_matrix = X_train.corr()

mask = np.triu(np.ones(corr_matrix.shape), k=1).astype(bool)
upper_corr = corr_matrix.where(mask)

threshold = 0.95
high_corr_pairs = (
    upper_corr
    .stack()                                 
    .rename("corr")                         
    .reset_index()                         
    .query("abs(corr) >= @threshold")       
)
high_corr_pairs.columns = ["feature_1", "feature_2", "corr_value"]

In [8]:
high_corr_pairs.sort_values(by=['corr_value'], ascending=False)

,feature_1,feature_2,corr_value
26787,CRB_mean14,CRB_mean21,0.995931
457,BDI_7,BDI_mean14,0.994987
25233,CRB_mean7,CRB_mean14,0.994020
24452,CRB_lag7,CRB_mean14,0.993897
85725,USD_mean14,USD_mean21,0.993769
...,...,...,...
15984,HY-IG Spread7,HY-IG Spread14,0.951134
101215,Russell_pct14,SP400_pct14,0.950977
15570,HY-IG Spread,HY-IG Spread7,0.950693
899,BDI_14,BDI_21,0.950176


In [9]:
def select_redundant_features(high_corr_pairs, threshold=0.9):
    to_remove = []
    for _, row in high_corr_pairs.iterrows():
        f1, f2, corr = row['feature_1'], row['feature_2'], row['corr_value']
        if abs(corr) < threshold:
            continue
        if f1 not in to_remove and f2 not in to_remove:
            to_remove.append(f2)
    return to_remove

<font size=4, font color='blue'> For pairs with correlation>0.95, drop one of them

In [10]:
to_drop = select_redundant_features(high_corr_pairs, threshold=0.95)
to_drop

['BDI_mean7',
 'BDI_mean14',
 'BDI_mean21',
 'BDI_21',
 'BDI_mean42',
 'HY-IG Spread',
 'Investment Grade OAS7',
 'HY-IG Spread14',
 'Investment Grade OAS21',
 'CRB_lag7',
 'CRB_mean7',
 'CRB_mean14',
 'CRB_mean21',
 'CRB_lag21',
 'CRB_mean42',
 'level_diff21',
 'EPU_mean21',
 'Industry_Production_Index1',
 'MOVE_mean7',
 'MOVE_mean21',
 'MOVE7',
 'P/C_Index_mean21',
 'VIX_mean14',
 'VVIX_mean21',
 'SPY_vol_mean21',
 'USD_mean7',
 'USD_mean14',
 'USD_mean21',
 'USD_mean42',
 'TPU_mean21',
 'oil_vol_mean21',
 'SP400_pct7',
 'SP400_pct14']

In [11]:
X_train = X_train[[col for col in X_train.columns.tolist() if col not in to_drop]]
X_test = X_test[[col for col in X_test.columns.tolist() if col not in to_drop]]

<font size=4, font color='blue'> Inspect VIF of remaining features

In [29]:
X = X_train.ffill()

vif_df = pd.DataFrame()
vif_df['feature'] = X.columns
vif_df['VIF'] = [
    variance_inflation_factor(X.values, i)
    for i in range(X.shape[1])
]

vif_df['high_vif'] = vif_df['VIF'] > 10

vif_df = vif_df.sort_values('VIF', ascending=False).reset_index(drop=True)

print(vif_df)

              feature         VIF  high_vif
0               10 Yr  697.141283      True
1        10 Yr_diff21  594.878544      True
2    Bull_Bear_Spread  553.858877      True
3               level  448.664949      True
4                5 Yr  418.201320      True
..                ...         ...       ...
413   investment_pct7    9.860303     False
414   employment_pct7    9.421336     False
415      savings_pct7    8.883958     False
416     shopping_pct7    8.483926     False
417         Recession    6.023088     False

[418 rows x 3 columns]


<font size=4, font color='blue'> Recursive Feature Elimination using a LightGBM model's feature importance scores

In [12]:
def recursive_rfe(
    X, y,
    eliminate_per_round: int = 20,
    target_features: int = 100,
    lgbm_params: dict = None,
    random_state: int = 42
):
    if lgbm_params is None:
        lgbm_params = {
          'objective':        'binary',
          'is_unbalance':     True,
          'n_estimators':     100,
          'learning_rate':    0.1,
          'subsample':        0.8,
          'colsample_bytree': 0.7,
          'subsample_freq':   1,
          'reg_alpha':        0.0,
          'reg_lambda':       0.5,
          'random_state':     random_state,
          'verbose':          -1,
          'n_jobs':           -1
        }

    history = []
    X_curr = X.copy()
    while X_curr.shape[1] > target_features:
        model = lgb.LGBMClassifier(**lgbm_params)
        model.fit(X_curr, y)
        
        imp_df = pd.DataFrame({
            'feature': X_curr.columns,
            'importance': model.feature_importances_
        }).set_index('feature').sort_values('importance', ascending=False)
        history.append(imp_df)

        n_drop = min(eliminate_per_round, X_curr.shape[1] - target_features)
        to_drop = imp_df.tail(n_drop).index.tolist()

        X_curr = X_curr.drop(columns=to_drop)
        print(f"Dropped {len(to_drop)} features; {X_curr.shape[1]} remain.")

    model = lgb.LGBMClassifier(**lgbm_params)
    model.fit(X_curr, y)
    final_imp = pd.DataFrame({
        'feature': X_curr.columns,
        'importance': model.feature_importances_
    }).set_index('feature').sort_values('importance', ascending=False)
    history.append(final_imp)

    return X_curr.columns.tolist(), history


In [19]:
selected_feats, rfe_history = recursive_rfe(
    X_train.ffill(), y_train,
    eliminate_per_round=5,
    target_features=40,
    random_state=42
)

print("Final features:", selected_feats)

Dropped 5 features; 413 remain.
Dropped 5 features; 408 remain.
Dropped 5 features; 403 remain.
Dropped 5 features; 398 remain.
Dropped 5 features; 393 remain.
Dropped 5 features; 388 remain.
Dropped 5 features; 383 remain.
Dropped 5 features; 378 remain.
Dropped 5 features; 373 remain.
Dropped 5 features; 368 remain.
Dropped 5 features; 363 remain.
Dropped 5 features; 358 remain.
Dropped 5 features; 353 remain.
Dropped 5 features; 348 remain.
Dropped 5 features; 343 remain.
Dropped 5 features; 338 remain.
Dropped 5 features; 333 remain.
Dropped 5 features; 328 remain.
Dropped 5 features; 323 remain.
Dropped 5 features; 318 remain.
Dropped 5 features; 313 remain.
Dropped 5 features; 308 remain.
Dropped 5 features; 303 remain.
Dropped 5 features; 298 remain.
Dropped 5 features; 293 remain.
Dropped 5 features; 288 remain.
Dropped 5 features; 283 remain.
Dropped 5 features; 278 remain.
Dropped 5 features; 273 remain.
Dropped 5 features; 268 remain.
Dropped 5 features; 263 remain.
Dropped 

In [20]:
sorted(rfe_history[-1].index.tolist()) == sorted(selected_feats)

True

<font size=4, font color='blue'> Expanding window purged CV and Optuna (Bayesian optimizer) for hyperparameter optimization

In [21]:
def expanding_window_cv(X, y, init_frac, val_frac, n_folds, lgb_params, random_state=42):
    n = len(y)
    init_window = int(np.floor(init_frac * n))
    val_window  = int(np.floor(val_frac  * n))
    aucs = []

    for fold in range(n_folds):
        train_end = init_window + fold * val_window
        val_start = train_end
        val_end   = val_start + val_window
        if val_end > n:
            break

        X_tr, y_tr = X[:train_end-63], y[:train_end-63]
        X_val, y_val = X[val_start:val_end], y[val_start:val_end]

        clf = lgb.LGBMClassifier(random_state=random_state, **lgb_params)
        clf.fit(X_tr, y_tr)
        y_proba = clf.predict_proba(X_val)[:, 1]
        aucs.append(roc_auc_score(y_val, y_proba))

    return np.mean(aucs)


def objective(trial):

    max_depth = trial.suggest_int('max_depth', 5, 15)
    max_leaves = min(2 ** max_depth, 200)
    num_leaves = trial.suggest_int('num_leaves', 20, max_leaves)


    param = {
        'objective': 'binary',
        'is_unbalance': True,
        'n_estimators': trial.suggest_int('n_estimators', 50, 500),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.3),
        'num_leaves': num_leaves,
        'max_depth': max_depth,
        'subsample': trial.suggest_float('subsample', 0.7, 0.9),
        'subsample_freq': 1,
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 0.9),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.01, 10.0),
        'n_jobs': -1
    }
    
    mean_auc = expanding_window_cv(
        X_train[selected_feats].ffill(),
        y_train,
        init_frac=0.30,
        val_frac=0.14,
        n_folds=5,
        lgb_params=param,
        random_state=42
    )
    return mean_auc


study = optuna.create_study(
    direction="maximize",
    sampler=optuna.samplers.TPESampler(seed=42)
)
study.optimize(objective, n_trials=200)

print("Best AUC:", study.best_value)
print("Best params:", study.best_params)

[I 2025-07-30 05:17:36,692] A new study created in memory with name: no-name-fcfad5f4-7c1d-4113-ab8a-b8cc004a2511
[I 2025-07-30 05:17:40,061] Trial 0 finished with value: 0.7362616009036904 and parameters: {'max_depth': 9, 'num_leaves': 192, 'n_estimators': 380, 'learning_rate': 0.17999888677491394, 'subsample': 0.7312037280884873, 'colsample_bytree': 0.6467983561008608, 'reg_lambda': 0.5902552855603126}. Best is trial 0 with value: 0.7362616009036904.
[I 2025-07-30 05:17:47,174] Trial 1 finished with value: 0.6727233710278003 and parameters: {'max_depth': 14, 'num_leaves': 128, 'n_estimators': 369, 'learning_rate': 0.0071547637944449315, 'subsample': 0.8939819704323989, 'colsample_bytree': 0.8497327922401265, 'reg_lambda': 2.1312677156759787}. Best is trial 0 with value: 0.7362616009036904.
[I 2025-07-30 05:17:48,786] Trial 2 finished with value: 0.764451799102102 and parameters: {'max_depth': 7, 'num_leaves': 39, 'n_estimators': 187, 'learning_rate': 0.1579021730580391, 'subsample': 

Best AUC: 0.7759674107629702
Best params: {'max_depth': 15, 'num_leaves': 138, 'n_estimators': 332, 'learning_rate': 0.281393510469453, 'subsample': 0.7545781389603842, 'colsample_bytree': 0.6602598373388076, 'reg_lambda': 3.870660430860955}


In [24]:
def expanding_window_cv(X, y,
                        init_frac=0.3,
                        val_frac=0.14,
                        n_folds=5,
                        random_state=42):
   
    n = len(y)
    init_window = int(np.floor(init_frac * n))
    val_window  = int(np.floor(val_frac  * n))

    accuracies = []
    aucs       = []

    lgbm_params = {
        'objective':        'binary',
        'is_unbalance':     True,
        'n_estimators':     332,
        'learning_rate':    0.281393510469453,
        'num_leaves':       138,
        'max_depth':        15,
        'subsample':        0.7545781389603842,
        'colsample_bytree': 0.6602598373388076,
        'subsample_freq':   1,
        'reg_alpha':        0.0,
        'reg_lambda':       3.870660430860955,
        'random_state':     random_state,
        'verbose':          -1,
        'n_jobs':           -1
    }

    for fold in range(n_folds):
        train_end = init_window + fold * val_window
        val_start = train_end
        val_end   = val_start + val_window
        if val_end > n:
            break  

        X_tr = X[:train_end-63]
        y_tr = y[:train_end-63]
        X_val = X[val_start:val_end]
        y_val = y[val_start:val_end]

        clf = lgb.LGBMClassifier(**lgbm_params)
        clf.fit(X_tr, y_tr)

        y_pred  = clf.predict(X_val)
        y_proba = clf.predict_proba(X_val)[:, 1]
        acc = accuracy_score(y_val, y_pred)
        auc = roc_auc_score(y_val, y_proba)

        accuracies.append(acc)
        aucs.append(auc)

        print(f"Fold {fold+1:2d} — train up to {train_end:4d}, "
              f"valid [{val_start:4d}:{val_end:4d}]  "
              f"→ Acc: {acc:.4f}, AUC: {auc:.4f}")

    mean_acc = np.mean(accuracies)
    mean_auc = np.mean(aucs)
    print(f"\nOverall CV → Mean Accuracy: {mean_acc:.4f}, Mean AUC: {mean_auc:.4f}")

    return mean_acc, mean_auc

mean_accuracy, mean_auc = expanding_window_cv(
    X_train[selected_feats].ffill(),
    y_train
)

Fold  1 — train up to  930, valid [ 930:1364]  → Acc: 0.6682, AUC: 0.7160
Fold  2 — train up to 1364, valid [1364:1798]  → Acc: 0.7074, AUC: 0.7863
Fold  3 — train up to 1798, valid [1798:2232]  → Acc: 0.7696, AUC: 0.7697
Fold  4 — train up to 2232, valid [2232:2666]  → Acc: 0.6544, AUC: 0.7681
Fold  5 — train up to 2666, valid [2666:3100]  → Acc: 0.6313, AUC: 0.8397

Overall CV → Mean Accuracy: 0.6862, Mean AUC: 0.7760


<font size=4, font color='blue'> Holdout test set performance (model refitted on the complete training set)

In [28]:
model = lgb.LGBMClassifier(
        objective='binary',
        is_unbalance=True,
        n_estimators=332,
        learning_rate= 0.281393510469453,
        num_leaves=138,
        max_depth=15,
        subsample=0.7545781389603842,
        colsample_bytree= 0.6602598373388076,
        subsample_freq=1,
        reg_alpha=0.0,
        reg_lambda=3.870660430860955,
        random_state=42,
        verbose=-1,
        n_jobs=-1
)

model.fit(
    X_train[selected_feats].ffill().iloc[:-63], y_train[:-63],
    eval_metric='binary_logloss',
)

y_pred = model.predict(X_test[selected_feats])
y_proba = model.predict_proba(X_test[selected_feats])[:, 1]

auroc = roc_auc_score(y_test, y_proba)
print(f'Test AUROC: {auroc:.4f}')

print("Test Accuracy: ", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

importances = pd.Series(model.feature_importances_, index=X_train[selected_feats].columns)
print("\nTop 10 features by importance:")
print(importances.sort_values(ascending=False).head(10))

Test AUROC: 0.7105
Test Accuracy:  0.663659793814433
              precision    recall  f1-score   support

           0       0.63      0.81      0.71       395
           1       0.72      0.51      0.60       381

    accuracy                           0.66       776
   macro avg       0.68      0.66      0.65       776
weighted avg       0.68      0.66      0.66       776


Top 10 features by importance:
P/C_Index_pct21      614
n_gas_vol_pct14      556
TPU_pct42            541
10Y-2Y_diff21        526
BDI_pct21            494
soybean_std7         482
VIX_std21            472
Russell_vol_std42    471
shopping_pct14       457
soybean_vol_std14    455
dtype: int32


<font size=4, font color='blue'> All feature importances

In [30]:
importances.sort_values(ascending=False)

,0
P/C_Index_pct21,614
n_gas_vol_pct14,556
TPU_pct42,541
10Y-2Y_diff21,526
BDI_pct21,494
soybean_std7,482
VIX_std21,472
Russell_vol_std42,471
shopping_pct14,457
soybean_vol_std14,455
